In [ ]:
import torch
import numpy as np
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [ ]:
class TwoLayersPerceptron(torch.nn.Module):
    def __init__(self, number_of_inputs=2, hidden_layer_size=3, number_of_outputs=2,
                 dropout_ratio=0.5):
        super(TwoLayersPerceptron, self).__init__()
        self.number_of_inputs = number_of_inputs
        self.hidden_layer_size = hidden_layer_size
        self.number_of_outputs = number_of_outputs
        self.dropout_ratio = dropout_ratio

        self.linear1 = torch.nn.Linear(self.number_of_inputs, self.hidden_layer_size)
        self.linear2 = torch.nn.Linear(self.hidden_layer_size, self.number_of_outputs)
        self.dropout = torch.nn.Dropout(p=self.dropout_ratio)

    def forward(self, input_tsr):  # input_tsr.shape = (N, N_in):
        act1 = self.linear1(input_tsr)  # (N, H)
        act2 = torch.nn.functional.relu(act1)  # (N, H)
        act3 = self.dropout(act2)  # (N, H)
        act4 = self.linear2(act3)  # (N, N_out)
        return act4

In [ ]:
# >>> Define a four-layers perceptron
class FourLayersPerceptron(torch.nn.Module):
    def __init__(self, number_of_inputs=2, hidden_layer1_size=100, hidden_layer2_size=100, 
                 hidden_layer3_size=100, number_of_outputs=2,
                 dropout_ratio=0.5):
        super(FourLayersPerceptron, self).__init__()
        pass

    def forward(self, input_tsr):  # input_tsr.shape = (N, N_in):
        pass

In [ ]:
# A class to store a list of features and a class index, from a pandas.core.frame.DataFrame
class FeaturesAndClass(Dataset):
    def __init__(self, dataset_filepath):
        super(FeaturesAndClass, self).__init__()
        self.dataset_df = pd.read_csv(dataset_filepath)

    def __len__(self):
        return len(self.dataset_df)

    def __getitem__(self, idx):
        observation = self.dataset_df.iloc[idx]  # Retrieve the observation
        features = list(observation[0: -1])  # List of features
        input_tsr = torch.tensor(features)  # Tensor of features
        class_tensor = torch.tensor(int(observation['class']))  # A tensor containing either 0 or 1
        return input_tsr, class_tensor  # Returns the input tensor and the target class index, as a tuple

In [ ]:
# Load the dataset
batch_size = 16
dataset_filepath = "/usercode/images/datasets/sixfeatures_threeclasses.csv"
dataset = FeaturesAndClass(dataset_filepath)
# Split the dataset into a training and a validation datasets
number_of_validation_observations = round(0.2 * len(dataset))
# >>> Split the dataset into train and validation datasets
train_dataset, validation_dataset = None, None
# >>> Create data loaders
train_dataloader = None
validation_dataloader = None

In [ ]:
# Create the neural network
# >>> Choose one of the two defined architectures (comment the other one)
hidden_layer_size = 100
dropout_ratio = 0.5
neural_net = TwoLayersPerceptron(
    number_of_inputs=6,
    hidden_layer_size=hidden_layer_size,
    number_of_outputs=3,
    dropout_ratio=dropout_ratio)

"""hidden_layer1_size = 100
hidden_layer2_size = 100
hidden_layer3_size = 100
dropout_ratio = 0.25
neural_net = FourLayersPerceptron(
    number_of_inputs=6,
    hidden_layer1_size=hidden_layer1_size,
    hidden_layer2_size=hidden_layer2_size,
    hidden_layer3_size=hidden_layer3_size,
    number_of_outputs=3,
    dropout_ratio=dropout_ratio)
"""

In [ ]:
# Training parameters
learning_rate = 0.003
weight_decay = 0.0000001
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(neural_net.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [ ]:
# Utility function
def numberOfCorrectPredictions(predictions_tsr, target_class_tsr):
    return sum(torch.argmax(predictions_tsr, dim=1) == target_class_tsr).item()

In [ ]:
# Training loop
# Record statistics
epochs = []
train_losses = []
validation_losses = []
accuracies = []

number_of_epochs = 50
for epoch in range(1, number_of_epochs + 1):
    # Set the neural network to training mode
    neural_net.train()
    running_loss = 0.0
    number_of_batches = 0
    for input_tsr, target_class_tsr in train_dataloader:
        # >>> Write the steps that must happen in a batch training loop:
        # >>> Set the parameter gradients to zero before every batch
        
        # >>> # Pass the input tensor through the neural network
        
        # >>> Compute the loss, i.e., the error function we want to minimize
        
        # >>> Backpropagate the loss function, to compute the gradient of the loss function with
        # respect to every trainable parameter in the neural network
        
        # >>> Perturb every trainable parameter by a small quantity, in the direction of the steepest loss descent
        
        # >>> Increment the running loss and the number of batches
        
        pass
    average_training_loss = running_loss/number_of_batches
    
    # Evaluate with the validation dataset
    # Set the neural network to evaluation (inference) mode
    neural_net.eval()
    validation_running_loss = 0.0
    number_of_batches = 0
    number_of_correct_predictions = 0
    number_of_predictions = 0
    for validation_input_tsr, validation_target_output_tsr in validation_dataloader:
        # >>> Write the steps that must happen in a batch validation loop:
        # >>> Pass the input tensor through the neural network
        
        # >>> Compute the validation loss
        
        # >>> Increment validation running loss, the number of correct predictions, 
        # the number of predictions, and the number of batches
        
        pass
    average_validation_loss = validation_running_loss/number_of_batches
    accuracy = number_of_correct_predictions/number_of_predictions
    print(f"Epoch {epoch}: average_training_loss = {average_training_loss}; average_validation_loss = {average_validation_loss}; accuracy = {accuracy}")
    epochs.append(epoch)
    train_losses.append(average_training_loss)
    validation_losses.append(average_validation_loss)
    accuracies.append(accuracy)

In [ ]:
# Display the metrics evolution
fig1, ax1 = plt.subplots()
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')
ax1.plot(epochs, train_losses, color='b', label='Training loss')
ax1.plot(epochs, validation_losses, color='r', label='Validation loss')
ax1.grid(True)
ax1.legend(loc='right')
ax2 = ax1.twinx()  # Instantiate a second axes that shares the same x-axis
ax2.set_ylabel('accuracy', color='g')
ax2.plot(epochs, accuracies, color='g', label='Accuracy')
ax2.legend(loc='upper right')
plt.show()